In [1]:
import numpy as np
import pandas as pd
import torch
from importlib import reload
from tqdm.auto import tqdm
from pathlib import Path
import utils
reload(utils)


2024-03-12 20:40:40.128613: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 20:40:40.128710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 20:40:40.293118: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


<module 'utils' from '/kaggle/usr/lib/utils/utils.py'>

In [2]:
BASE_PATH = Path("/kaggle/input/hms-harmful-brain-activity-classification")
SPEC_DIR = Path("/tmp/spectrograms_npy") # where to save the spectrograms after converting to numpy
utils.BASE_PATH = BASE_PATH
utils.SPEC_DIR = SPEC_DIR

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
utils.DEVICE = DEVICE
print(DEVICE)


cuda


In [3]:
# Define model:
model = utils.Spectrogram_EfficientNet(frozen=False, download=False)

# Load model from here:
MODEL_PATH = "/kaggle/input/spectrogram-efficientnetb0/pytorch/unfrozen-5000-items/2/spectrogram_effnetb0_unfrozen_5000_items.pt"
model.load_state_dict(torch.load(MODEL_PATH))
model.to(DEVICE);

## Everything below this point should be runnable without changes

In [4]:
metadata_test = utils.metadata_df("test")
metadata_test
utils.create_spec_npy_dirs()
for spec_id in metadata_test.spectrogram_id:
    utils.process_spec(spec_id, split="test")

Created directory /tmp/spectrograms_npy/train_spectrograms
Created directory /tmp/spectrograms_npy/test_spectrograms


In [5]:
metadata_test

,spectrogram_id,eeg_id,patient_id,eeg_path,spec_path,spec_npy_path
0,853520,3911565283,6885,/kaggle/input/hms-harmful-brain-activity-class...,/kaggle/input/hms-harmful-brain-activity-class...,/tmp/spectrograms_npy/test_spectrograms/853520...


In [6]:
from utils import SpectrogramTestDataset
from torch.utils.data import DataLoader
test_dset = SpectrogramTestDataset(metadata_test)
test_loader = DataLoader(test_dset, batch_size=32, shuffle=False)

In [7]:
all_preds = []
model.eval()
for batch in test_loader:
    with torch.inference_mode():
        preds = model(batch[0].to(DEVICE))
    preds = torch.softmax(preds, dim=1).cpu().numpy()
    all_preds.append(preds)

all_preds = np.vstack(all_preds)
print(all_preds)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


[[1.6331840e-04 2.0428864e-02 6.1101623e-02 1.5760459e-01 5.0435700e-02
  7.1026599e-01]]


In [8]:
preds_df = pd.DataFrame(columns=["seizure_vote", "lpd_vote", "gpd_vote",
                                 "lrda_vote", "grda_vote", "other_vote"],
                        index=metadata_test.eeg_id,
                        data=all_preds)
preds_df.to_csv("submission.csv")